In [4]:
import pandas as pd
from sqlalchemy import create_engine

# Your database credentials
DATABASE_TYPE = 'postgresql'
DB_USER = 'postgres'  # Your PostgreSQL username
DB_PASSWORD = 'BestPasswordEver'  # Your PostgreSQL password
DB_HOST = '34.133.41.198'  # External IP of your VM or database host
DB_PORT = '5432'  # Default PostgreSQL port
DB_NAME = 'postgres'  # Database name

# Create an SQLAlchemy engine
engine = create_engine(f'{DATABASE_TYPE}://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

# Dictionary to hold DataFrames for each table
dataframes = {}

# Connect to the database and verify the connection
try:
    with engine.connect() as connection:
        print("Connected to PostgreSQL")

        # Fetch all table names from the current database
        table_query = """
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = 'public'
        """
        
        # Load the table names into a DataFrame
        table_names = pd.read_sql(table_query, connection)

        # Iterate through the table names and load each table into a DataFrame
        for table_name in table_names['table_name']:
            query = f"SELECT * FROM {table_name}"
            dataframes[table_name] = pd.read_sql(query, connection)
            print(f"Loaded table: {table_name} with {dataframes[table_name].shape[0]} rows.")

except Exception as e:
    print(f"An error occurred: {e}")

# Now you can access each DataFrame by its table name in the `dataframes` dictionary
# Example: dataframes['Property'] to access the Property table DataFrame

# If you want to see the data of all tables
for name, df in dataframes.items():
    print(f"\nData from table: {name}")

Connected to PostgreSQL
Loaded table: location with 9083 rows.
Loaded table: agency with 5924 rows.
Loaded table: agent with 11353 rows.
Loaded table: property with 191393 rows.

Data from table: location

Data from table: agency

Data from table: agent

Data from table: property


In [11]:
location = dataframes['location']
property_df.head()

,location_id,city,province_name,locality,latitude,longitude
0,8,Lahore,Punjab,"Model Town, Lahore, Punjab",31.483869,74.325686
1,48,Lahore,Punjab,"Multan Road, Lahore, Punjab",31.431593,74.179980
2,75,Lahore,Punjab,"Eden, Lahore, Punjab",31.499348,74.416959
3,3821,Lahore,Punjab,"Gulberg, Lahore, Punjab",31.522069,74.355512
4,3522,Lahore,Punjab,"Allama Iqbal Town, Lahore, Punjab",31.506483,74.286017


In [12]:
agency = dataframes['agency']
agency.head()

,agency,agency_id
0,Real Biz International,1
1,Khan Estate,2
2,Shahum Estate 2,3
3,None,4
4,MATZ Services,5


In [13]:
property = dataframes['property']
property.head()

,property_id,property_type,price,price_bin,area,area_marla,area_sqft,baths,bedrooms,purpose,location_id,date_added,agency_id,agent_id
0,347795,House,220000000,Very High,6 Kanal,120.0,32670.12,0,0,For Sale,8,2019-07-17,1,1
1,482892,House,40000000,Very High,1 Kanal,20.0,5445.02,5,5,For Sale,48,2018-10-06,2,2
2,555962,House,9500000,Low,9 Marla,9.0,2450.26,0,3,For Sale,75,2019-07-03,3,3
3,562843,House,125000000,Very High,1 Kanal,20.0,5445.02,7,8,For Sale,3821,2019-04-04,4,4
4,686990,House,21000000,High,11 Marla,11.0,2994.76,5,6,For Sale,3522,2019-04-04,4,4


In [14]:
# Count the number of properties in each province
province_counts = location['province_name'].value_counts().reset_index()
province_counts.columns = ['province_name', 'property_count']

print("Property Counts by Province:")
print(province_counts)

# Count the number of properties in each city
city_counts = property_df['city'].value_counts().reset_index()
city_counts.columns = ['city', 'property_count']

print("\nProperty Counts by City:")
print(city_counts)

Property Counts by Province:
       province_name  property_count
0             Punjab            4725
1              Sindh            2765
2  Islamabad Capital            1593

Property Counts by City:
         city  property_count
0      Lahore            3080
1     Karachi            2765
2   Islamabad            1593
3  Rawalpindi            1024
4  Faisalabad             621
